In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


+ create layer, shapefile, polygon, crs 26918  with filename unique.
+ toggle editing
+ add polygon 
+ draw around outside of unique tiles
" de-toggle editing
+ save edits

In [20]:
import os

def check_path(fp):
    if not os.path.exists(fp):
        #return
        print("missing: ", fp)
        os.makedirs(fp)
        if not os.path.exists(fp):
            print("still missing: ", fp)
        else:
            print("made directory: ", fp)

#batch_group = "pa_northcentral_2019_b19"
#batch_group = "pa_allentown_2016"
#batch_group = "pa_3_county_south_central_2018_d18"
#batch_group = "pa_luzernecounty_2018"
#batch_group = "pa_sandy_2014"
#batch_group = "pa_south_central_b1_2017"
#batch_group = "pa_south_central_b2_2017"
#batch_group = "pa_dauphin_2016"
#batch_group = "pa_westernpa_2019_d20"
batch_group = "de_delawarevalley_hd_2015"
#batch_group ="md-pa_sandysupp_2014"

dem_tif_fp = os.path.join("/content/drive/MyDrive/crane_pennsylvania/dem/",batch_group)
check_path(dem_tif_fp)
overlapped_dem_tif_fp = os.path.join(dem_tif_fp,"overlapped/")
check_path(overlapped_dem_tif_fp)
prediction_fp = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_'+batch_group+'/'
#dem_tif_fp, slope_tif_fp, slope_tif_tiles_fp, slope_tif_tiles_jpgs_fp

import os
def check_number_of_files(folder_path, file_extension):
    counter = 0
    for filename in os.listdir(folder_path):
        ext = filename[-1*(len(file_extension)):]
        if(ext == file_extension):
            counter = counter + 1
            # print(filename, counter)
    print("found: ",counter," files in ", folder_path)



missing:  /content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/overlapped/
made directory:  /content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/overlapped/


# Optional: Remove duplicate tiles. 
Some tiles overlap

In [15]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
def find_overlap(shape_to_be_clipped_fp, shape_overlap_fp, polys_to_remove_df):
    print(len(polys_to_remove_df))

    if os.path.exists(shape_to_be_clipped_fp) and os.path.exists(shape_overlap_fp):
        shape_to_be_clipped_polys = gpd.read_file(shape_to_be_clipped_fp)
        print("Total dem polys:", len(shape_to_be_clipped_polys), " in: ", shape_to_be_clipped_fp, shape_overlap_fp)

        shape_overlap_polys = gpd.read_file(shape_overlap_fp)
        print("crs", shape_to_be_clipped_polys.crs,shape_overlap_polys.crs )
        print("shape_to_be_clipped_polys",shape_to_be_clipped_polys.shape, "shape_overlap_polys",shape_overlap_polys.shape)
        shape_overlap_geoseries = shape_overlap_polys['geometry']

        matched_tiles_polys = list()    

        row_counter = 0 
        for index, row in shape_to_be_clipped_polys.iterrows():

           check_poly = row[2]
           if not (shape_overlap_geoseries.contains(check_poly).any()==True) or (shape_overlap_geoseries.within(check_poly).any()==True):
               #print(row_counter, "match", row[0])
               polys_to_remove_df = polys_to_remove_df.append(row, ignore_index=True)
           #else:
               #print(row_counter, "    no match", row[0])  
               row_counter = row_counter + 1 
    else:
        print("Does not exist:",shape_to_be_clipped_fp, shape_overlap_fp)
    return polys_to_remove_df


In [26]:
# make a layer of just polygons inside forested areas

import os
import geopandas as gpd

crs = 26918
dem_shape_to_be_clipped_fp = os.path.join(prediction_fp,(batch_group+"_"+str(crs)+"_dem_boundary.shp"))

dem_shape_to_remove_fp = os.path.join(prediction_fp,(batch_group+"_"+str(crs)+"_dem_to_remove_boundary.shp"))

# Create an empty geopandas GeoDataFrame for predictions in a forested area
dems_to_remove_df = gpd.GeoDataFrame()
dems_to_remove_df['geometry'] = None
dems_to_remove_df.crs = ('EPSG:26918')

dem_shape_overlap_fp = "/content/drive/MyDrive/crane_pennsylvania/pa_michaux_forest_area_rch/dems_borders_of_unique_tiles/"+batch_group+"_26918_dem_unique.shp"
dems_to_remove_df = find_overlap(dem_shape_to_be_clipped_fp, dem_shape_overlap_fp, dems_to_remove_df)
print("Total dems to be removed:", len(dems_to_remove_df))

if not dems_to_remove_df.empty:
    dems_to_remove_df.to_file(dem_shape_to_remove_fp)
    print("file in in",dem_shape_to_remove_fp)
    print("Total dems to be removed:", len(dems_to_remove_df))

else:
    print("empty")

0
Total dem polys: 111  in:  /content/drive/MyDrive/crane_pennsylvania/predictions/project_de_delawarevalley_hd_2015/de_delawarevalley_hd_2015_26918_dem_boundary.shp /content/drive/MyDrive/crane_pennsylvania/pa_michaux_forest_area_rch/dems_borders_of_unique_tiles/de_delawarevalley_hd_2015_26918_dem_unique.shp
crs epsg:26918 epsg:26918
shape_to_be_clipped_polys (111, 3) shape_overlap_polys (1, 2)
Total dems to be removed: 4
file in in /content/drive/MyDrive/crane_pennsylvania/predictions/project_de_delawarevalley_hd_2015/de_delawarevalley_hd_2015_26918_dem_to_remove_boundary.shp
Total dems to be removed: 4


In [27]:
# Move overlapped files
import shutil

if os.path.exists(overlapped_dem_tif_fp) and os.path.exists(dem_shape_to_remove_fp):
    tifs_to_move_polys = gpd.read_file(dem_shape_to_remove_fp)
    print("Total dems to moved:", len(tifs_to_move_polys))
    row_counter = 0 
    for index, row in tifs_to_move_polys.iterrows():
        file_to_move = row[0]
        print(file_to_move)
        file_to_move_from_fp = os.path.join(dem_tif_fp,file_to_move)
        file_to_move_to_fp = os.path.join(overlapped_dem_tif_fp,file_to_move)
        print(file_to_move_from_fp, file_to_move_to_fp)
        shutil.move(file_to_move_from_fp, file_to_move_to_fp)
else:
    print("does not exist:",overlapped_dem_tif_fp,dem_shape_to_remove_fp)  

Total dems to moved: 4
USGS_one_meter_x44y446_DE_DelawareValley_HD_2015.tif
/content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/USGS_one_meter_x44y446_DE_DelawareValley_HD_2015.tif /content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/overlapped/USGS_one_meter_x44y446_DE_DelawareValley_HD_2015.tif
USGS_one_meter_x44y447_DE_DelawareValley_HD_2015.tif
/content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/USGS_one_meter_x44y447_DE_DelawareValley_HD_2015.tif /content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/overlapped/USGS_one_meter_x44y447_DE_DelawareValley_HD_2015.tif
USGS_one_meter_x45y447_DE_DelawareValley_HD_2015.tif
/content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/USGS_one_meter_x45y447_DE_DelawareValley_HD_2015.tif /content/drive/MyDrive/crane_pennsylvania/dem/de_delawarevalley_hd_2015/overlapped/USGS_one_meter_x45y447_DE_DelawareValley_HD_2015.tif
USGS_one_meter_x45y448_DE_DelawareV

In [10]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 5.4 MB/s 


In [29]:
#The good one
import rasterio
import pandas as pd
import geopandas as gpd

import os
from os import listdir
df = ""

def create_geodef(crs, dem_tif_fp): 
    df = gpd.GeoDataFrame()
    df['geometry'] = None
    df.crs = ("EPSG:" + str(crs))
    df.geometry = df.geometry.to_crs(crs)
    df.to_crs(crs)
    df = df.to_crs(epsg=crs)

    counter = 0
    for dem_filename in listdir(dem_tif_fp):

        if(dem_filename[-4:])==".tif":
            ra = rasterio.open(os.path.join(dem_tif_fp,dem_filename))
            ra_crs = ra.crs
            print(dem_filename, ra_crs)
            if int(str(ra_crs)[5:])==crs:
                bounds  = ra.bounds
                from shapely.geometry import box
                geom = box(*bounds)
                new_row = {'id':counter, "filename":dem_filename,'geometry':geom}
                df = df.append(new_row, ignore_index=True)
                #print(df)
                counter = counter + 1
    df = df.set_crs(epsg=crs)
    df_fp = os.path.join(prediction_fp,(batch_group+"_"+str(crs)+"_dem_boundary.shp"))
    df.to_file(df_fp)
    print("the file is in ",df_fp,counter)


df_29618 = create_geodef(26918, dem_tif_fp)
#df_29617 = create_geodef(26917, dem_tif_fp)

USGS_one_meter_x44y441_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x44y442_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x45y441_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x45y442_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x45y444_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x45y445_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x45y446_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y440_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y441_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y442_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y443_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y444_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y445_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y446_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y447_DE_DelawareValley_HD_2015.tif EPSG:26918
USGS_one_meter_x46y448_DE_DelawareValley

In [43]:
check_number_of_files(dem_tif_fp, ".tif")

found:  70  files in  /content/drive/MyDrive/crane_pennsylvania/dem/md-pa_sandysupp_2014
